# Learning to Drive in Adverse Weather Conditions

## Pre-Notes

- Only got access to lab computers on 04/11/21 (Week 8)
- Had to update version of Firefox as github would not work correctly
- No sudo access so could only install things that were packaged or had their own installer

## Week 1

### Setup Carla & Anaconda

- Carla 0.9.12 was installed on the machines by IT in /opt little setup required just figuring out configs and some crashing errors
- Anaconda was installed from a package, Quite easy and quick 

### Install Carla Gym

- Installing Carla Gym not too hard, API not very well documented so had dig through code files
- Some Issues as pygame only works with python <= 3.7, overcame this with anaconda env (there is a 3.8 version but it requires a sudo install)
- Had to update some code do to older version of Carla being used (See Adapt Carla Env)

### Additional Libraries

- interp e2e driving installed fine but crashed after a number of runs due to low memory
- d4rl would not install due to mujuco dependency (May be free now)
- didn't try d3rlpy ye

### Adapt Carla Env

### First Experiment - Minimize Distance to Waypoint